# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
raw_df = pd.read_csv('../WeatherPy/Output/raw_df.csv', index_col = 0)
raw_df.head()



,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Pangnirtung,66.15,-65.71,-8.00,72,40,1.50,CA,1603651903
1,Santa Fe,-31.63,-60.70,25.56,77,8,1.34,AR,1603651904
2,Sitka,57.05,-135.33,4.00,80,90,3.10,US,1603651905
3,Qaanaaq,77.48,-69.36,-10.80,80,100,5.56,GL,1603651893
4,Iquitos,-3.75,-73.25,35.00,44,75,4.10,PE,1603651908


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
#Configure gmaps
gmaps.configure(api_key=g_key)


In [4]:
#Use Lat and Lng as locaiton and Humidity as the weight
#raw_df.info()
humidity = raw_df["Humidity"].astype(float)
humidity.head()

locations= raw_df[["Lat","Lng"]]
locations.head()



,Lat,Lng
0,66.15,-65.71
1,-31.63,-60.70
2,57.05,-135.33
3,77.48,-69.36
4,-3.75,-73.25


In [5]:
fig_layout = gmaps.figure( 
                    display_toolbar = True,
                    display_errors = True,
                    zoom_level = 2,
                    tilt = 45,
                    center = (0,30),
                    layout = None,
                    map_type = 'ROADMAP',
                    mouse_handling ='COOPERATIVE', )
fig_layout

heat_layer = gmaps.heatmap_layer(locations, 
                                 weights = humidity, 
                                 dissipating = False, 
                                 max_intensity = 100,
                                 point_radius = 3)

fig_layout.add_layer(heat_layer)

heat_map = fig_layout

heat_map

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [17]:
#Max Temp lower that 80 but higher than 70
#Wind speed less than 10mph
#Zero cloudiness
#First must convert C to F
#80F
#((80-32)*(5/9)) = 26.67
#70F
#((70-32)*(5/9)) = 21.11

narrow_df = (raw_df.loc[(raw_df["Max Temp"] < 26.67) & 
                        (raw_df["Max Temp"] > 21.11) & 
                        (raw_df["Wind Speed"] < 10) & 
                        (raw_df["Cloudiness"] == 0)])

narrow_df = narrow_df.dropna(how = 'any')
narrow_df.reset_index()

narrow_df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
17,Cidreira,-30.18,-50.21,21.75,82,0,9.47,BR,1603651926
25,Atuona,-9.80,-139.03,26.07,74,0,9.09,PF,1603651887
37,Charters Towers,-20.10,146.27,22.35,50,0,4.18,AU,1603651953
49,Faanui,-16.48,-151.75,26.62,69,0,1.79,PF,1603651970
77,Bourail,-21.57,165.48,21.79,81,0,1.95,NC,1603652012


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [ ]:
hotel_df = narrow_df.loc[:, ["City", "Country", 'Lat", "Lng"]]

In [ ]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [ ]:
# Add marker layer ontop of heat map


# Display figure
